In [1]:
import numpy as np
import copy
from tqdm import tqdm
import pandas as pd
import re
import gensim

from keras import optimizers
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.layers import *
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from score import report_score
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers
from tensorflow.keras import regularizers

c:\ProgramData\Anaconda3\envs\keras\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
datadir="fnc-1"
w2v_path = './w2v/GoogleNews-vectors-negative300.bin'
save_path = "./saved/"
batch_size = 128
max_sent_length = 250
random_state = 37
lstm_hidden_dim = 100
epoch = 20

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
raw_train_bodies = pd.read_csv(datadir + '/train_bodies.csv')   
raw_train_stances = pd.read_csv(datadir + '/train_stances.csv')
raw_test_bodies = pd.read_csv(datadir + '/competition_test_bodies.csv') 
raw_test_stances = pd.read_csv(datadir + '/competition_test_stances.csv') 

In [5]:
stance_to_int = {"agree":0, "discuss": 1, "disagree": 2, "unrelated": 3}
int_to_stance = {0:"agree", 1:"discuss", 2:"disagree", 3: "unrelated"}

In [6]:
actual_test_stances = raw_test_stances['Stance']
raw_train_stances['Stance'] = raw_train_stances['Stance'].apply(lambda x: stance_to_int[x])
raw_test_stances['Stance'] = raw_test_stances['Stance'].apply(lambda x: stance_to_int[x])

In [7]:
train_df = raw_train_stances.join(raw_train_bodies.set_index('Body ID'), on='Body ID')
test_df = raw_test_stances.join(raw_test_bodies.set_index('Body ID'), on='Body ID')

In [8]:
def clean(s):
    # Cleans a string: Lowercasing, trimming, removing non-alphanumeric
    return " ".join(re.findall(r'\w+', s, flags=re.UNICODE)).lower()

# Pre-processing words
clean_train_headline = [text_to_word_sequence(clean(head)) for head in train_df['Headline']]
clean_train_bodies = [text_to_word_sequence(clean(body)) for body in train_df['articleBody']]
clean_test_headline = [text_to_word_sequence(clean(head)) for head in test_df['Headline']]
clean_test_bodies = [text_to_word_sequence(clean(body)) for body in test_df['articleBody']]

In [9]:
wordlist = []
for i in range(len(clean_train_headline)):
    wordlist.append(clean_train_headline[i])
for i in range(len(clean_train_bodies)):
    wordlist.append(clean_train_bodies[i])
for i in range(len(clean_test_headline)):
    wordlist.append(clean_test_headline[i])
for i in range(len(clean_test_bodies)):
    wordlist.append(clean_test_bodies[i])

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(wordlist)
len(tokenizer.word_index)

29451

In [11]:
train_lines = [] 
for i in range(len(clean_train_headline)):
    headline =  clean_train_headline[i]
    body = clean_train_bodies[i]
    newline = headline+body
    train_lines.append(newline)

test_lines = [] 
for i in range(len(clean_test_headline)):
    headline =  clean_test_headline[i]
    body = clean_train_bodies[i]
    newline = headline+body
    test_lines.append(newline)

In [12]:
X_train = tokenizer.texts_to_sequences([' '.join(seq[:max_sent_length]) for seq in train_lines])
raw_X_train = pad_sequences(X_train, maxlen=max_sent_length, padding='post', truncating='post')
raw_y_train = train_df['Stance']

In [13]:
X_test = tokenizer.texts_to_sequences([' '.join(seq[:max_sent_length]) for seq in test_lines])
X_test = pad_sequences(X_test, maxlen=max_sent_length, padding='post', truncating='post')
y_test = test_df['Stance']

In [14]:
# Convert y to onehot
y_train_onehot = np_utils.to_categorical(raw_y_train)
y_test_onehot = np_utils.to_categorical(y_test)

### Basic Model

In [15]:
embedding_dim = 300
embeddings = gensim.models.KeyedVectors.load_word2vec_format(w2v_path, binary=True)
embeddings_matrix = np.random.uniform(-0.05, 0.05, size=(len(tokenizer.word_index)+1, embedding_dim))

for word, i in tokenizer.word_index.items():
    try:
        embeddings_vector = embeddings[word]
        embeddings_matrix[i] = embeddings_vector
    except KeyError:
        pass
        
del embeddings


In [16]:
def lstm_model(n_classes):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index)+1,
                            output_dim=embedding_dim,
                            weights = [embeddings_matrix],
                            trainable=False, name='embedding_layer',
                            mask_zero=True))

    model.add(Bidirectional(LSTM(lstm_hidden_dim, return_sequences=False, name='lstm_layer',
                    
                    kernel_regularizer =tf.keras.regularizers.L2(l2=1e-3))))
    model.add(Dropout(rate=0.8, name='dropout'))
    model.add(Activation(activation='relu', name='activation_1'))
    model.add(Dense(n_classes, activation='softmax', name='output_layer'))
    return model

### Basic model trained over to four-classfier

In [17]:
basic_model = lstm_model(n_classes=4)
basic_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [18]:
print(basic_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, None, 300)         8835600   
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               320800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
output_layer (Dense)         (None, 4)                 804       
Total params: 9,157,204
Trainable params: 321,604
Non-trainable params: 8,835,600
_________________________________________________________________
None


In [19]:
history = basic_model.fit(raw_X_train, y_train_onehot,
          batch_size=batch_size,
          epochs=epoch)
# basic_model.save(save_path+"basic_mode")

Epoch 1/20
391/391 [==============================] - 29s 52ms/step - loss: 0.9062 - accuracy: 0.7330
Epoch 2/20
391/391 [==============================] - 20s 52ms/step - loss: 0.7229 - accuracy: 0.7497
Epoch 3/20
391/391 [==============================] - 20s 51ms/step - loss: 0.6388 - accuracy: 0.7738
Epoch 4/20
391/391 [==============================] - 20s 52ms/step - loss: 0.6031 - accuracy: 0.7884
Epoch 5/20
391/391 [==============================] - 20s 52ms/step - loss: 0.5625 - accuracy: 0.8050
Epoch 6/20
391/391 [==============================] - 21s 53ms/step - loss: 0.5515 - accuracy: 0.8119
Epoch 7/20
391/391 [==============================] - 21s 54ms/step - loss: 0.5284 - accuracy: 0.8226
Epoch 8/20
391/391 [==============================] - 21s 55ms/step - loss: 0.5050 - accuracy: 0.8311
Epoch 9/20
391/391 [==============================] - 21s 54ms/step - loss: 0.4871 - accuracy: 0.8387
Epoch 10/20
391/391 [==============================] - 21s 55ms/step - loss: 0.471

In [20]:
preds = basic_model.predict(X_test)
outputs = [int_to_stance[np.argmax(p, axis = -1)] for p in preds]
report_score(actual_test_stances,outputs)

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    65     |     0     |    53     |   1785    |
-------------------------------------------------------------
| disagree  |    16     |     0     |    19     |    662    |
-------------------------------------------------------------
|  discuss  |    89     |     0     |    278    |   4097    |
-------------------------------------------------------------
| unrelated |    471    |     0     |    940    |   16938   |
-------------------------------------------------------------
Score: 4621.75 out of 11651.25	(39.667417659049455%)


39.667417659049455